In [1]:
from trainer import pl_model

/home/chidhambararajan/docVuVenv/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/chidhambararajan/docVuVenv/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


(1, 1, 149, 64)
torch.Size([1, 2354]) (1, 1, 149, 64) (1, 149, 64)


Global seed set to 69
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: theseriousprogrammer. Use `wandb login --relogin` to force relogin


Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [2]:
import torch

In [4]:
for name, param in pl_model.pytorch_model.named_parameters() :
    print(name, param.shape)

mapper torch.Size([1152, 1750])
feature_network.layers.0.weight torch.Size([1])
feature_network.layers.0.bias torch.Size([1])
feature_network.layers.1.0.weight torch.Size([16, 1, 3, 3])
feature_network.layers.1.1.weight torch.Size([16])
feature_network.layers.1.1.bias torch.Size([16])
feature_network.layers.2.conv.0.weight torch.Size([16, 16, 3, 3])
feature_network.layers.2.conv.1.weight torch.Size([16])
feature_network.layers.2.conv.1.bias torch.Size([16])
feature_network.layers.2.conv.4.weight torch.Size([16, 16, 1, 1])
feature_network.layers.2.conv.5.weight torch.Size([16])
feature_network.layers.2.conv.5.bias torch.Size([16])
feature_network.layers.3.conv.0.weight torch.Size([36, 16, 1, 1])
feature_network.layers.3.conv.1.weight torch.Size([36])
feature_network.layers.3.conv.1.bias torch.Size([36])
feature_network.layers.3.conv.3.weight torch.Size([36, 36, 3, 3])
feature_network.layers.3.conv.4.weight torch.Size([36])
feature_network.layers.3.conv.4.bias torch.Size([36])
feature_ne

In [4]:
import torch.nn as nn
for module in pl_model.pytorch_model.modules():
    #print(module)
    #if isinstance(module, nn.Parameter) :
    #    print(module.data)
    #continue    
    if not hasattr(module, "weight") :
        continue
    print(module, torch.linalg.norm(module.weight/torch.linalg.norm(module.weight)))
    print(module, torch.mean(module.weight/torch.linalg.norm(module.weight)), torch.var(module.weight/torch.linalg.norm(module.weight)))
    continue
    if isinstance(module, nn.Conv2d):
    
        for iteration in range(1):
            print(module.weight.shape)
            weight_mean = module.weight.mean(dim=1, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            std = torch.sqrt(torch.var(module.weight.view(module.weight.size(0), -1), dim=1) + 1e-12).view(-1, 1, 1, 1) + 1e-5
    
            module.weight = torch.nn.Parameter(module.weight - weight_mean)
            module.weight = torch.nn.Parameter(module.weight / std.expand_as(module.weight))

BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) tensor(1., grad_fn=<LinalgVectorNormBackward0>)
BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) tensor(1., grad_fn=<MeanBackward0>) tensor(nan, grad_fn=<VarBackward0>)
Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False) tensor(1.0000, grad_fn=<LinalgVectorNormBackward0>)
Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False) tensor(-0.0021, grad_fn=<MeanBackward0>) tensor(0.0070, grad_fn=<VarBackward0>)
BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) tensor(1., grad_fn=<LinalgVectorNormBackward0>)
BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) tensor(0.2500, grad_fn=<MeanBackward0>) tensor(0., grad_fn=<VarBackward0>)
Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) tensor(1., grad_fn=<LinalgVectorNormBackward0>)
Conv2d(16, 16, kernel_size=

In [7]:
pl_model.pytorch_model.mapper.data = nn.functional.normalize(pl_model.pytorch_model.mapper.data, dim=1)